In [1]:
import pymongo
import json
import pandas as pd

from pymongo import MongoClient

client = MongoClient('mongo', 27017, username='admin', password='DataMan2019!')

db=client.UCLFinal

In [11]:
agg=db.pre_partita.aggregate([
   {'$match':{"$and":[{"timestamp":{"$gt":"2020-08-23 19:24:00","$lte":"2020-08-23 21:00:00"}}, 
    {"text":{'$regex': "[Nn]eymar"}}]}},  
     {
       '$project':
         {
             'id':'$id',
             
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet'
             
            }
     },
       { '$out' :'Ney_pre'}   ]   
   
)




In [21]:
agg=db.post_partita.aggregate([
   {'$match':{"$and":[{"timestamp":{"$gt":"2020-08-23 22:54:00","$lte":"2020-08-24 00:30:00"}},
    {"text":{'$regex': "[Nn]eymar"}}]}},  
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
            
            }
     },
       { '$out' :'Ney_post'}   ]   
   
)


In [23]:
pos=[]
neu=[]
neg=[]
punteggio=[]
stdev=[]
lb=[]
ub=[]
stdev=[]
confint_score=[]


 #si creano queste liste vuote

In [32]:
#SENTIMENT ANALYSIS:

collezione = db.Ney_post.find({"language":"en"})   #inseriamo tutti e due i subset e compiliamo i dataset 

import re
import math
import numpy as np

from textblob import TextBlob

positive =0 
negative =0
neutral=0
totale=0

lista=[]

def clean_tweet(tweet): 
    ''' Utility function to clean tweet text by removing links, special characters using simple regex statements. ''' 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+://\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet):

    global positive
    global negative
    global neutral
    global totale
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(clean_tweet(tweet)) 
    # set sentiment 
    #print(analysis.sentiment.polarity)
    if analysis.sentiment.polarity > 0:
        lista.append(analysis.sentiment.polarity)
        positive=positive+1
        #print(analysis)
        totale=totale+analysis.sentiment.polarity
        return 'positive'

    elif analysis.sentiment.polarity == 0: 
        neutral=neutral+1
        #print(analysis)
        return 'neutral'
    else: 
        lista.append(analysis.sentiment.polarity)
        negative=negative+1
        totale=totale+analysis.sentiment.polarity
        #print(analysis)
        return 'negative'

for tweet in collezione:

    get_tweet_sentiment(tweet['text'])
    

#creo 4 liste da comporre in un dataset finale:

punt=totale/(positive+negative)
dev=math.sqrt(np.var(lista))
lower=punt-1.64*dev/(math.sqrt(positive+negative)) #intervallo normale
upper=punt+1.64*dev/(math.sqrt(positive+negative))
confint=(round(lower,3),round(upper,3))

pos.append(positive)
neu.append(neutral)
neg.append(negative)
punteggio.append(punt)
lb.append(lower)
ub.append(upper)
stdev.append(dev)
confint_score.append(confint)


In [ ]:
Ney_pre=pd.DataFrame({'time':'prepartita','positivi':pos,'negativi':neg,'neutrali':neu,'punteggio':punteggio,'confint_score':confint_score})
Ney_pre["sentiment_score"]=round(Ney_pre["punteggio"],3)
Ney_pre=Ney_pre.drop("punteggio",axis='columns')
Ney_pre.to_csv("Ney_pre",sep='\t',index=False)

In [ ]:
Ney_post=pd.DataFrame({'time':'postpartita','positivi':pos,'negativi':neg,'neutrali':neu,'punteggio':punteggio,'confint_score':confint_score})
Ney_post["sentiment_score"]=round(Ney_post["punteggio"],3)
Ney_post=Ney_post.drop("punteggio",axis='columns')
Ney_post.to_csv("Ney_post",sep='\t',index=False)

In [ ]:
# WORD CLOUD NEYMAR

In [ ]:
#girare json in csv:

pre_df = pd.DataFrame(list(db.Ney_pre.find()))  #csv prepartita

post_df = pd.DataFrame(list(db.Ney_post.find()))  #csv postpartita


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.extend(['#UCL','#ChampionsLeague','#PSGBayern','#UCLFinal','#PSGFCB'])
#lista di stop word inglesi + hastags

In [ ]:
pre_df["text"]=pre_df["text"].str.lower().str.split()
pre_df["text"]=pre_df["text"].apply(lambda x: [item for item in x if item not in stop])
pre_df.to_csv("Neymar_prepartita_fin.csv",sep=';')
#file csv da aprire su excel e lavorarci in maniera tale da ottenere cloud word

In [ ]:
post_df["text"]=post_df["text"].str.lower().str.split()
post_df["text"]=post_df["text"].apply(lambda x: [item for item in x if item not in stop])
post_df.to_csv("Neymar_postpartita_fin.csv",sep=';')
#file csv postpartita da aprire su excel e creare cloud word post